<a id="intro"></a>
# <b><span style='color:#006241;font-size:300%'>1 |</span><span style='color:#006241;font-size:300%'> INTRODUCTION</span></b>

<a id="about"></a>
# <b><span style='color:#006241;font-size:150%'>1-1 |</span><span style='color:#006241;font-size:150%'> About the kernel</span></b>

Following tips and tricks for optimizing black box (BB) function, hyperparamter, etc. with [Optuna][1] in Kaggle environment are summarized with code snippets in the kernel,
- [Run preliminary few fixed parameters trials for reflecting prior knowledge](#pre_trials) (ex. Run trials whose parameters (x, y) = (0.1, 0.1)(parameters close to minimum) when you have prior knowledge about search space, run trials whose parameters (x, y) = (-5, 5), (5, 5), (-5, 5), (-5, -5) to get information about whole search space roughly),
- [Run additional fixed number of trials](#2nd_10_trials) (ex. Run additional 10 trials, i.e. from 11 to 20 trials),
- [Run trials in fixed running time](#3rd_1_sec_trials) (ex. Run trials in 1 seconds, i.e. from 21 trials to ?? in 1 seconds),
- [Save optimization history and restart optimization](#restart),
- [Show optimized value and corresponding parameters](#best),
- [Plot optimization history](#history),
- [Plot contour of objective function based on optimization history](#contour),
- [Plot parameters importances for optimization](#importance),
- [Plot parameters relationship](#relation).

It may take long time to find information about the tips and tricks for Optuna starters/begginers, so it should be useful for them. Tips and tricks in the kernel are very simple and easy to understand. You can read all the contents within 10 minites or so. If you don't have enough time to read, you can jump to interested ones from the above/below links.

<b><div style='color:#D4E9E2;font-size:180%'>TIPS :</div></b>
<b><div style='color:#D4E9E2;font-size:120%'>- Optuna is an automatic hyperparameter optimization software framework, particularly designed for machine learning. It features an imperative, define-by-run style user API. Thanks to our define-by-run API, the code written with Optuna enjoys high modularity, and the user of Optuna can dynamically construct the search spaces for the hyperparameters.</div></b>

[1]: https://optuna.org/

<a id="table"></a>
# <b><span style='color:#006241;font-size:150%'>1-2 |</span><span style='color:#006241;font-size:150%'> Table of contents</span></b>

- [1. INTRODUCTION](#intro)
  - [1-1. About the kernel](#about)
  - [1-2. Table of contents](#table)
- [2. FUNCTION OPTIMIZATION](#opt)
  - [2-1. Import libraries](#lib)
  - [2-2. Define objective function](#obj)
  - [2-3. Run few preliminary trials for reflecting prior knowledge](#pre_trials)
  - [2-4. Run additional 10 trials](#2nd_10_trials)
  - [2-5. Run additional trials in 1 sec.](#3rd_1_sec_trials)
  - [2-6. Visualize optimization result](#viz)
  - [2-7. Save optimization history and restart optimization](#restart)
- [3. REFERENCES](#ref)
- [4. ADDITIONAL MATERIALS](#materials)

[1]: https://optuna.org/

<a id="opt"></a>
# <b><span style='color:#006241;font-size:300%'>2 |</span><span style='color:#006241;font-size:300%'> FUNCTION OPTIMIZATION</span></b>

Optimize (minimize) one of the following functions "Sphere function" with Optuna as an example,

## <b><div style='padding:20px;background-color:#F2F0EB;color:#000000;border-radius:5px;font-size:100%'>Sphere function</div></b>

$$ f(x, y) = x^2 + y^2 , $$

## <b><div style='padding:20px;background-color:#F2F0EB;color:#000000;border-radius:5px;font-size:100%'>Three-hump camel function</div></b>

$$ f(x, y) = 2x^2 - 1.05x^4 + \frac{x^6}{6} + xy + y^2 . $$

Those are commonly used for evaluating performance of optimization algorithm (You can find the other ones [here][2].). Minimum value for both functions is 0 when x=0, y=0. 

[2]: https://en.wikipedia.org/wiki/Test_functions_for_optimization

<a id="lib"></a>
# <b><span style='color:#006241;font-size:150%'>2-1 |</span><span style='color:#006241;font-size:150%'> Import libraries</span></b>

In [1]:
# Import required libs.
import numpy as np
import optuna
import pickle
import plotly.graph_objects as go
import itertools

<a id="obj"></a>
# <b><span style='color:#006241;font-size:150%'>2-2 |</span><span style='color:#006241;font-size:150%'> Define objective function</span></b>

## <b><div style='padding:20px;background-color:#F2F0EB;color:#000000;border-radius:5px;font-size:100%'>Utility functions for defining objective function</div></b>

In [2]:
# Define utility functions for defining objective function.
def objective(trial):
    """Define objective function for optimization with Optuna."""
    # this is randomly sampling
    x = trial.suggest_float(name="x", low=-5.0, high=5.0, step=0.1)
    y = trial.suggest_float(name="y", low=-5.0, high=5.0, step=0.1)
    return _sphereFunction(x=x, y=y) # It can be modified to _threeHumpCamelFunction(), if it is required.

def _sphereFunction(x, y):
    """Return x^2 + y^2. Minimum value is 0.0 when x=0.0, y=0.0."""
    return pow(x, 2) + pow(y, 2)

def _threeHumpCamelFunction(x, y):
    """Return 2 x^2 - 1.05 x^4 + x^6 / 6 + xy + y^2. Minimum value is 0.0 when x=0.0, y=0.0."""
    return pow(x, 2) * 2 - pow(x, 4) * 1.05 + pow(x, 6) / 6 + x*y + pow(y, 2)

def plotSurface(objective, x_range_step, y_range_step, eps=0.001):
    """Plot 3D surface of given function with Plotly."""
    # Prepare data for plot.
    start, stop, step = x_range_step
    x = np.arange(start, stop + eps, step)
    start, stop, step = y_range_step
    y = np.arange(start, stop + eps, step)
    z = np.array([[objective(optuna.trial.FixedTrial({"x": x_, "y": y_})) for x_ in x] for y_ in y])
    
    # Create figure object and plot.
    fig = go.Figure()
    trace = go.Surface(x=x, y=y, z=z,contours_z = dict(show=True, project_z=True))
    fig.add_trace(trace)
    fig.show()

## <b><div style='padding:20px;background-color:#F2F0EB;color:#000000;border-radius:5px;font-size:100%'>3D surface of objective function</div></b>

In [3]:
# Plot 3D surface of objective function for confirmation.
plotSurface(objective=objective, x_range_step=(-5, 5, 0.1), y_range_step=(-5, 5, 0.1))

<a id="pre_trials"></a>
# <b><span style='color:#006241;font-size:150%'>2-3 |</span><span style='color:#006241;font-size:150%'> Run few preliminary trials for reflecting prior knowledge</span></b>

Run preliminary few fixed trials for getting information about whole search space roughly. It may help to optimize function efficiently.

<b><div style='color:#D4E9E2;font-size:180%'>TIPS :</div></b>
<b><div style='color:#D4E9E2;font-size:120%'>- This way of running trials is suitable for kaggle, which running time for a kernel is limitted.</div></b>

## <b><div style='padding:20px;background-color:#F2F0EB;color:#000000;border-radius:5px;font-size:100%'>Utility functions for preliminary optimization</div></b>

In [4]:
# Define utility functions for optimizing function efficiently.
def addPreliminaryTrials(study, base_params, choices):
    """Add fixed trials for optimizing hyperparameters efficiently.
    The source code may difficult to understand for begginers, so it
    is recommended to check usage below first. It is also available
    for reflecting prior knowledge about search space."""
    updated_params = base_params.copy()
    
    # Create all combinations of given parameters.
    value_combinations = itertools.product(*list(choices.values()))
    
    # Add fixed trials for all combinations of parameters to given study object.
    for value_combination in value_combinations:
        # Update given parameters.
        for key, value in zip(choices.keys(), value_combination):
            updated_params[key] = value
        
        # Add a fixed trial.
        study.enqueue_trial(params=updated_params, skip_if_exists=True)
        
    return study

## <b><div style='padding:20px;background-color:#F2F0EB;color:#000000;border-radius:5px;font-size:100%'>Preliminary optimization</div></b>

In [5]:
# Create study for function optimization.
study = optuna.create_study()

[I 2023-04-06 15:49:26,775] A new study created in memory with name: no-name-26f69b53-04d2-436d-a11e-6b7feaf7b89d


In [6]:
# Add few preliminary fixed trials for optimizing function efficiently.
base_params = {
    "x": 1.0,
    "y": 1.0,
}
choices = {
    "x": [-5, -2, 2, 5],
    "y": [-5, -2, 2, 5],
}
study = addPreliminaryTrials(study=study, base_params=base_params, choices=choices)
study.trials

[FrozenTrial(number=0, state=TrialState.WAITING, values=None, datetime_start=None, datetime_complete=None, params={}, user_attrs={}, system_attrs={'fixed_params': {'x': -5, 'y': -5}}, intermediate_values={}, distributions={}, trial_id=0, value=None),
 FrozenTrial(number=1, state=TrialState.WAITING, values=None, datetime_start=None, datetime_complete=None, params={}, user_attrs={}, system_attrs={'fixed_params': {'x': -5, 'y': -2}}, intermediate_values={}, distributions={}, trial_id=1, value=None),
 FrozenTrial(number=2, state=TrialState.WAITING, values=None, datetime_start=None, datetime_complete=None, params={}, user_attrs={}, system_attrs={'fixed_params': {'x': -5, 'y': 2}}, intermediate_values={}, distributions={}, trial_id=2, value=None),
 FrozenTrial(number=3, state=TrialState.WAITING, values=None, datetime_start=None, datetime_complete=None, params={}, user_attrs={}, system_attrs={'fixed_params': {'x': -5, 'y': 5}}, intermediate_values={}, distributions={}, trial_id=3, value=None)

In [7]:
# Run preliminary fixed trials.
n_trials = len(study.trials) # number of preliminary fixed trials
study.optimize(objective, n_trials=n_trials)

[I 2023-04-06 15:49:26,846] Trial 0 finished with value: 50.0 and parameters: {'x': -5.0, 'y': -5.0}. Best is trial 0 with value: 50.0.
[I 2023-04-06 15:49:26,847] Trial 1 finished with value: 29.0 and parameters: {'x': -5.0, 'y': -2.0}. Best is trial 1 with value: 29.0.
[I 2023-04-06 15:49:26,847] Trial 2 finished with value: 29.0 and parameters: {'x': -5.0, 'y': 2.0}. Best is trial 1 with value: 29.0.
[I 2023-04-06 15:49:26,848] Trial 3 finished with value: 50.0 and parameters: {'x': -5.0, 'y': 5.0}. Best is trial 1 with value: 29.0.
[I 2023-04-06 15:49:26,848] Trial 4 finished with value: 29.0 and parameters: {'x': -2.0, 'y': -5.0}. Best is trial 1 with value: 29.0.
[I 2023-04-06 15:49:26,849] Trial 5 finished with value: 8.0 and parameters: {'x': -2.0, 'y': -2.0}. Best is trial 5 with value: 8.0.
[I 2023-04-06 15:49:26,849] Trial 6 finished with value: 8.0 and parameters: {'x': -2.0, 'y': 2.0}. Best is trial 5 with value: 8.0.
[I 2023-04-06 15:49:26,849] Trial 7 finished with value

## <b><div style='padding:20px;background-color:#F2F0EB;color:#000000;border-radius:5px;font-size:100%'>Preliminary optimization result/history</div></b>

In [8]:
# Show optimization result.
print(f"  Optimized value : {study.best_value}")
print(f"  Parameters : {study.best_params}")

  Optimized value : 8.0
  Parameters : {'x': -2.0, 'y': -2.0}


In [9]:
# Plot optimization history.
fig = optuna.visualization.plot_optimization_history(study)
fig.show()

In [10]:
# Plot contour of parameters.
fig = optuna.visualization.plot_contour(study, params=["x", "y"])
fig.show()

Ran preliminary few fixed trials. Optimized value is not still closer to minimum value (=0), but it was found that it may be closer to minimum value in the range -2 < x < 2, -2 < y < 2.

<a id="2nd_10_trials"></a>
# <b><span style='color:#006241;font-size:150%'>2-4 |</span><span style='color:#006241;font-size:150%'> Run additional 10 trials</span></b>

Run additional 10 trials for optimization. Then show optimization result/history for better understanding.

## <b><div style='padding:20px;background-color:#F2F0EB;color:#000000;border-radius:5px;font-size:100%'>Optimization</div></b>

In [11]:
# Run additional 10 trials. We can continue optimization just by calling optimize() using same study object. 
n_trials = 10
study.optimize(objective, n_trials=n_trials)

[I 2023-04-06 15:49:27,384] Trial 16 finished with value: 5.77 and parameters: {'x': -2.4, 'y': 0.10000000000000053}. Best is trial 16 with value: 5.77.
[I 2023-04-06 15:49:27,389] Trial 17 finished with value: 10.330000000000002 and parameters: {'x': -3.2, 'y': -0.2999999999999998}. Best is trial 16 with value: 5.77.
[I 2023-04-06 15:49:27,394] Trial 18 finished with value: 0.4099999999999996 and parameters: {'x': -0.39999999999999947, 'y': -0.5}. Best is trial 18 with value: 0.4099999999999996.
[I 2023-04-06 15:49:27,399] Trial 19 finished with value: 0.050000000000000176 and parameters: {'x': 0.10000000000000053, 'y': 0.20000000000000018}. Best is trial 19 with value: 0.050000000000000176.
[I 2023-04-06 15:49:27,403] Trial 20 finished with value: 0.04000000000000007 and parameters: {'x': 0.0, 'y': 0.20000000000000018}. Best is trial 20 with value: 0.04000000000000007.
[I 2023-04-06 15:49:27,408] Trial 21 finished with value: 0.25 and parameters: {'x': 0.0, 'y': 0.5}. Best is trial 2

## <b><div style='padding:20px;background-color:#F2F0EB;color:#000000;border-radius:5px;font-size:100%'>Optimization result/history</div></b>

In [12]:
# Show optimization result. The results of first 10 trials is also reflected to the result.
print(f"  Optimized value : {study.best_value}")
print(f"  Parameters : {study.best_params}")

  Optimized value : 0.04000000000000007
  Parameters : {'x': 0.0, 'y': 0.20000000000000018}


In [13]:
# Plot optimization history. The results of first 10 trials is also included in the figure.
fig = optuna.visualization.plot_optimization_history(study)
fig.show()

In [14]:
# Plot contour of parameters. The results of first 10 trials is also included in the figure.
fig = optuna.visualization.plot_contour(study, params=["x", "y"])
fig.show()

Ran additional 10 trials for optimization. Optimized value becomes significantly closer to minimum value (=0). We should run additional trials for further optimization.

<a id="3rd_1_sec_trials"></a>
# <b><span style='color:#006241;font-size:150%'>2-5 |</span><span style='color:#006241;font-size:150%'> Run additional trials in 1 seconds</span></b>

Run additional trials in 1 seconds (without specifing number of trials) for further optimization. Then show result/history of optimization for better understanding.

<b><div style='color:#D4E9E2;font-size:180%'>TIPS :</div></b>
<b><div style='color:#D4E9E2;font-size:120%'>- This way of running trials is suitable for kaggle, which running time for a kernel is limitted.</div></b>

## <b><div style='padding:20px;background-color:#F2F0EB;color:#000000;border-radius:5px;font-size:100%'>Optimization</div></b>

In [15]:
# Run additional trials in 1 seconds. We can continue optimization just by calling optimize() using same study object. 
timeout = 1.0
study.optimize(objective, timeout=timeout)

[I 2023-04-06 15:49:27,561] Trial 26 finished with value: 10.819999999999999 and parameters: {'x': 1.1000000000000005, 'y': -3.0999999999999996}. Best is trial 20 with value: 0.04000000000000007.
[I 2023-04-06 15:49:27,566] Trial 27 finished with value: 12.200000000000003 and parameters: {'x': 3.4000000000000004, 'y': 0.8000000000000007}. Best is trial 20 with value: 0.04000000000000007.
[I 2023-04-06 15:49:27,574] Trial 28 finished with value: 12.770000000000001 and parameters: {'x': -1.0999999999999996, 'y': 3.4000000000000004}. Best is trial 20 with value: 0.04000000000000007.
[I 2023-04-06 15:49:27,580] Trial 29 finished with value: 1.2800000000000007 and parameters: {'x': 0.8000000000000007, 'y': -0.7999999999999998}. Best is trial 20 with value: 0.04000000000000007.
[I 2023-04-06 15:49:27,586] Trial 30 finished with value: 2.6000000000000005 and parameters: {'x': -0.7999999999999998, 'y': 1.4000000000000004}. Best is trial 20 with value: 0.04000000000000007.
[I 2023-04-06 15:49:2

## <b><div style='padding:20px;background-color:#F2F0EB;color:#000000;border-radius:5px;font-size:100%'>Optimization result/history</div></b>

In [16]:
# Show optimization result. The results of previous 20 trials is also reflected to the result.
print(f"  Optimized value : {study.best_value}")
print(f"  Parameters : {study.best_params}")

  Optimized value : 0.0
  Parameters : {'x': 0.0, 'y': 0.0}


In [17]:
# Plot optimization history. The results of previous 20 trials is also included in the figure.
fig = optuna.visualization.plot_optimization_history(study)
fig.show()

In [18]:
# Plot contour of parameters. The results of previous 20 trials is also included in the figure.
fig = optuna.visualization.plot_contour(study, params=["x", "y"])
fig.show()

Ran additional 90 (or so) trials in 1 second for further optimization. Optimized value almost equals to minimum value (=0). Optimization is done.

<a id="viz"></a>
# <b><span style='color:#006241;font-size:150%'>2.6 |</span><span style='color:#006241;font-size:150%'> Visualize optimization result</span></b>

Visualize final optimization result. Show following data/figures here,
- [Optimized value and corresponding parameters](#best),
- [Optimization history (optimized value for each trial)](#history),
- [Contour of parameters (optimized value and corresponding parameters for each trial)](#contour),
- [Parameters importances](#importance),
- [Parameters relationship](#relation).

<a id="best"></a>
## <b><div style='padding:20px;background-color:#F2F0EB;color:#000000;border-radius:5px;font-size:100%'>Optimized value and corresponding parameters</div></b>

In [19]:
# Show final result of optimization.
print(f"  Optimized value : {study.best_value}")
print(f"  Parameters : {study.best_params}")

  Optimized value : 0.0
  Parameters : {'x': 0.0, 'y': 0.0}


<a id="history"></a>
## <b><div style='padding:20px;background-color:#F2F0EB;color:#000000;border-radius:5px;font-size:100%'>Optimization history</div></b>

In [20]:
# Plot optimization history (optimized value for each trial).
fig = optuna.visualization.plot_optimization_history(study)
fig.show()

<a id="contour"></a>
## <b><div style='padding:20px;background-color:#F2F0EB;color:#000000;border-radius:5px;font-size:100%'>Contour of parameters</div></b>

In [21]:
# Plot contour of parameters (optimized value and corresponding parameters for each trial).
fig = optuna.visualization.plot_contour(study, params=["x", "y"])
fig.show()

<a id="importance"></a>
## <b><div style='padding:20px;background-color:#F2F0EB;color:#000000;border-radius:5px;font-size:100%'>Parameters importances</div></b>

In [22]:
# Plot parameters importances. The importances of x and y are expected to be same value for the example,
# but those are different because of limited number of trials. Those will be closer if addtional trials are run.
fig = optuna.visualization.plot_param_importances(study)
fig.show()

<a id="relation"></a>
## <b><div style='padding:20px;background-color:#F2F0EB;color:#000000;border-radius:5px;font-size:100%'>Parameters relation</div></b>

In [23]:
# Plot parameters relation.
fig = optuna.visualization.plot_parallel_coordinate(study, params=["x", "y"])
fig.show()

<a id="restart"></a>
# <b><span style='color:#006241;font-size:150%'>2.7 |</span><span style='color:#006241;font-size:150%'> Save optimization history and restart optimization</span></b>

Save optimization history (study object) as pickle file. Then restart additional 1000 trials using saved study object.

NOTE : It is recommended to save study object to DB in [Optuna HP][1]. But above way of saving is better for Kaggle, since you don't need to prepare DB. You can continue optimization in 2 or more kernels using the above way.

## <b><div style='padding:20px;background-color:#F2F0EB;color:#000000;border-radius:5px;font-size:100%'>Utility functions for saving optimization history and restarting optimization</div></b>

In [24]:
# Define utility functions for saving optimization history and restart optimization.
def toPickle(obj, path_to_pickle):
    """Save obj as pickle file."""
    with open(path_to_pickle, "wb") as fout:
        pickle.dump(obj, fout)
        
def fromPickle(path_to_pickle):
    """Load obj from pickle file."""
    with open(path_to_pickle, "rb") as fin:
        return pickle.load(fin)

## <b><div style='padding:20px;background-color:#F2F0EB;color:#000000;border-radius:5px;font-size:100%'>Saving optimization history (study object)</div></b>

In [25]:
# Save optimization history (study object) as pickle file.
path_to_study = "/kaggle/working/study.pkl"
toPickle(obj=study, path_to_pickle=path_to_study)
!ls {path_to_study}

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/study.pkl'

## <b><div style='padding:20px;background-color:#F2F0EB;color:#000000;border-radius:5px;font-size:100%'>Restarting optimization</div></b>

In [ ]:
# Reload saved study object and restart optimization (Run additional 1000 trials. It is
# meaningless calculation, just an example for showing how to restart optimization.).
study_from_pickle = fromPickle(path_to_pickle=path_to_study)
n_trials = 1000
study_from_pickle.optimize(objective, n_trials=n_trials)

## <b><div style='padding:20px;background-color:#F2F0EB;color:#000000;border-radius:5px;font-size:100%'>Optimization result/history</div></b>

In [ ]:
# Show optimization result.
print(f"  Optimized value : {study_from_pickle.best_value}")
print(f"  Parameters : {study_from_pickle.best_params}")

In [ ]:
# Plot optimization history.
fig = optuna.visualization.plot_optimization_history(study_from_pickle)
fig.show()

In [ ]:
# Plot contour of parameters.
fig = optuna.visualization.plot_contour(study_from_pickle, params=["x", "y"])
fig.show()

In [ ]:
# Plot parameters importances. The importances of x and y are closer as expected.
fig = optuna.visualization.plot_param_importances(study_from_pickle)
fig.show()

<a id="ref"></a>
# <b><span style='color:#006241;font-size:300%'>3 |</span><span style='color:#006241;font-size:300%'> REFERENCES</span></b>

The kernel was created with reference to the following materials,
- [Test functions for optimization (Wiki)][2],
- [OPTUNA, Optimize Your Optimization, An open source hyperparameter optimization framework to automate hyperparameter search (Optuna HP)][1],
- [1. Lightweight, versatile, and platform agnostic architecture (in Optuna tutorial)][3],
- [5. Quick Visualization for Hyperparameter Optimization Analysis (in Optuna tutorial)][4],
- [optuna.study.Study.optimize(func[, n_trials, timeout, n_jobs, ...]) (in Optuna API reference)][5],
- [Saving/Resuming Study with RDB Backend (in Optuna tutorial)][6],
- [How can I save and resume studies? (in Optuna FAQ)][7].

[1]: https://optuna.org/
[2]: https://en.wikipedia.org/wiki/Test_functions_for_optimization
[3]: https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/001_first.html#sphx-glr-tutorial-10-key-features-001-first-py
[4]: https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#sphx-glr-tutorial-10-key-features-005-visualization-py
[5]: https://optuna.readthedocs.io/en/stable/reference/generated/optuna.study.Study.html#optuna.study.Study.optimize
[6]: https://optuna.readthedocs.io/en/v2.0.0/tutorial/rdb.html
[7]: https://optuna.readthedocs.io/en/stable/faq.html#how-can-i-save-and-resume-studies

<a id="materials"></a>
# <b><span style='color:#006241;font-size:300%'>4 |</span><span style='color:#006241;font-size:300%'> ADDITIONAL MATERIALS</span></b>

The following materials may be useful for better understanding of Optuna,
- [Optuna: A Next-generation Hyperparameter Optimization Framework (White paper)][8],
- [How do I avoid running out of memory (OOM) when optimizing studies? (in Optuna FAQ)][9],
- [How can I obtain reproducible optimization results? (in Optuna FAQ)][10],
- [Multi-objective Optimization with Optuna (in Optuna tutorial)][11],
- [Specify Hyperparameters Manually (in Optuna tutorial)][12].

[8]: https://arxiv.org/abs/1907.10902
[9]: https://optuna.readthedocs.io/en/stable/faq.html#how-do-i-avoid-running-out-of-memory-oom-when-optimizing-studies
[10]: https://optuna.readthedocs.io/en/stable/faq.html#how-can-i-obtain-reproducible-optimization-results
[11]: https://optuna.readthedocs.io/en/stable/tutorial/20_recipes/002_multi_objective.html#sphx-glr-tutorial-20-recipes-002-multi-objective-py
[12]: https://optuna.readthedocs.io/en/stable/tutorial/20_recipes/008_specify_params.html#sphx-glr-tutorial-20-recipes-008-specify-params-py

<b><div style='padding:20px;background-color:#006241;color:white;border-radius:5px;font-size:700%'>Thank you for reading!!</div></b>